In [1]:
%matplotlib qt5
%load_ext autoreload
%autoreload 2
import os, sys, fnmatch
import cv2, time
import numpy as np
import argparse, pprint


from matplotlib import pyplot as plt
import matplotlib
from matplotlib.ticker import NullFormatter
from matplotlib.transforms import Bbox
import matplotlib.gridspec as gridspec

import ipywidgets as widgets
from IPython.display import display
from ipywidgets import interact, interactive, IntSlider, Layout, interact_manual,interact, HBox, Layout,VBox

pp = pprint.PrettyPrinter(indent=4)
homeDir = os.path.abspath('')

from src.UVMappingPipeline import UVMappingPipeline
from src.VBOATS import VBOATS
from utils.sys_utils import *
from utils.seg_utils import *
from utils.uv_mapping_utils import *

In [2]:
pipe = UVMappingPipeline()
vboat = VBOATS()

In [ ]:
# tdir = "/home/hunter/data/images/obstacles"
tdir = "/home/hunter/data/images/test_cases"
# tdir = "/home/hyoung/data/images/obstacles"
# tdir = homeDir + "/test/iai/obstacles"

found_imgs, _ = grab_dir_images(tdir)

In [ ]:
%reload_ext autoreload
plt.close('all')

p = interactive(pipe.pipeline,
    _img = widgets.Select(
                        options=sorted(found_imgs),
                        value=str(sorted(found_imgs)[0]),
                        description='File:',
                        layout=Layout(width='100%')
                    ),
    line_method = {'Standard Hough Transform':0,'Probabilistic Hough Transform':1},
    input_method = {'Classic Thresholded':0,'Morphed Thresholded':1,'Canny Edges':2},
    e1 = (1,255,1), e2 = (1,255,1),
    ang = (0,360,1), rho = (1,1000,1),
    minLineLength = (0,255,1), maxLineGap = (0,255,1),
    houghThresh = (0,255,1), greyThreshU = (0,255,1), greyThreshV = (0,255,1), cnt_thresh = (0,5000,100),
    show_helpers = True, use_umap = True, flip_thresh_bin = False, norm_grey_thresh = (0.0,1.0,0.01)
)


row1 = HBox([p.children[0]])
row2 = HBox([p.children[1], p.children[2]], layout = Layout(flex_flow='row wrap'))
row3 = HBox([p.children[3], p.children[4], p.children[9]], layout = Layout(flex_flow='row wrap'))
row4 = HBox([p.children[5], p.children[6], p.children[7], p.children[8]], layout = Layout(display='flex',flex_flow='row'))

row5 = HBox([p.children[10], p.children[11], p.children[12], ], layout = Layout(flex_flow='row wrap'))
row6 = HBox([p.children[16]], layout = Layout(flex_flow='row wrap'))
row7 = HBox([p.children[13], p.children[14], p.children[15]], layout = Layout(flex_flow='row wrap'))
controls = VBox([row1,row2,row3,row4,row5,row6,row7], layout = Layout(display='flex'))
output = p.children[-1]
display(VBox([controls, output]))

In [ ]:
tmp1 = np.copy(pipe.vmap); tmp0 = cv2.cvtColor(tmp1,cv2.COLOR_GRAY2BGR)

normImg = (tmp1 - np.mean(tmp1))/np.std(tmp1)

dmax_norm = np.max(normImg)
dmin_norm = np.min(normImg)
test = cv2.normalize(normImg,None,0,1,cv2.NORM_MINMAX)
m = 255.0/(dmax_norm-dmin_norm)
print(dmin_norm,dmax_norm,m)
tmp = cv2.convertScaleAbs(test, test)#, m)#,np.mean(tmp))
print(np.min(tmp),np.max(tmp),np.mean(tmp))

fig1 = plt.figure(6)
fig1.canvas.set_window_title('Custom Filter Input')
plt.imshow(pipe.vmap,cmap='gray')
plt.show()

iters = 3; kSz = (1,6)
thresh = [5, 10, 20, 30]
filters = [cv2.THRESH_BINARY,cv2.THRESH_BINARY_INV,cv2.THRESH_TRUNC+cv2.THRESH_BINARY_INV,cv2.THRESH_TOZERO]
kernelC = cv2.getStructuringElement(cv2.MORPH_CROSS,kSz)
kernelE = cv2.getStructuringElement(cv2.MORPH_RECT,(1,15))

_, grey = cv2.threshold(tmp1,5,255,filters[0])#+cv2.THRESH_TRUNC)

# gradient = cv2.morphologyEx(grey, cv2.MORPH_CLOSE, kSz)
gradient = cv2.morphologyEx(grey, cv2.MORPH_OPEN, kSz)

# tmpKernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,kSz)
tmpKernel = cv2.getStructuringElement(cv2.MORPH_RECT,kSz)

gradient2 = cv2.erode(tmp0,kernelE,iterations = 1)
gradient2 = cv2.dilate(gradient2,tmpKernel,iterations = 1)
# preMorph = cv2.erode(grey,kernelE,iterations = 3)
# preMorph = cv2.dilate(grey,tmpKernel,iterations = 3)


# # morph = cv2.morphologyEx(grey,cv2.MORPH_CLOSE,(1,2), iterations = 2)
# morph = cv2.morphologyEx(grey,cv2.MORPH_OPEN,(2,3), iterations = 1)

# _,preMorph = cv2.threshold(preMorph,1,255,cv2.THRESH_BINARY_INV)
# tmpKernel = cv2.getStructuringElement(cv2.MORPH_RECT,(3,15))
# preMorph = cv2.dilate(preMorph,tmpKernel,iterations = 3)
# tmpKernel = cv2.getStructuringElement(cv2.MORPH_RECT,(10,3))
# preMorph = cv2.erode(preMorph,kernelE,iterations = 2)
# print(preMorph.shape)

# mask = cv2.cvtColor(preMorph,cv2.COLOR_BGR2GRAY)
# res = cv2.bitwise_and(tmp,tmp, mask=mask)
_, res = cv2.threshold(gradient2,5,255,cv2.THRESH_BINARY)

out = res

fig2 = plt.figure(7)
fig2.canvas.set_window_title('Custom Filter Output')
plt.imshow(out,cmap='gray')
plt.show()

In [ ]:
print("""
POTENTIALLY GOOD VMAP FILTERING
""")

tmp1 = np.copy(pipe.vmap); tmp0 = cv2.cvtColor(tmp1,cv2.COLOR_GRAY2BGR)

kSz = (1,6)
kernelE = cv2.getStructuringElement(cv2.MORPH_RECT,(1,15))
_, grey = cv2.threshold(tmp1,5,255,cv2.THRESH_BINARY)

gradient = cv2.morphologyEx(grey, cv2.MORPH_OPEN, kSz)
tmpKernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,kSz)

gradient2 = cv2.erode(tmp0,kernelE,iterations = 1)
gradient2 = cv2.dilate(gradient2,tmpKernel,iterations = 1)
_, res = cv2.threshold(gradient2,5,255,cv2.THRESH_BINARY)

out = res

In [ ]:
# print(pipe.disparityBounds[obsId],pipe.xBounds[obsId])

tmp = np.copy(pipe.greyV)
tmp2 = np.copy(pipe.img)
tmp2 = cv2.cvtColor(tmp2,cv2.COLOR_GRAY2BGR)
nObs = len(pipe.disparityBounds)
print(nObs)
b = 2; d = np.ceil(nObs / float(b))

plt.close(7)
fig, axes = plt.subplots(nrows=b, ncols=int(d), num=7)
obstacleImgs = []
for i in range(nObs):
    tmp2 = np.copy(pipe.img)
    tmp2 = cv2.cvtColor(tmp2,cv2.COLOR_GRAY2BGR)
    try:
        xs = pipe.xBounds[i]
        ys,disp = pipe.find_obstacles(tmp,pipe.disparityBounds[i], _threshold=30, lower_threshold=1, verbose = True)
        cv2.rectangle(tmp2,(xs[0],ys[0]),(xs[1],ys[-1]),(150,0,0),1)
    except:
        pass
    fig.axes[i].imshow(disp)
    obstacleImgs.append(tmp2)
plt.show()

In [ ]:
im = np.copy(pipe.img)
norm = (im - np.mean(im))/np.std(im)
h, w = norm.shape[:2]

umapOrig, vmapOrig,_ = pipe.get_uv_map(im,get_overlay=False)

plt.figure(0)
plt.imshow(im, cmap='gray')
plt.show()
plt.figure(1)
plt.imshow(norm, cmap='gray')
plt.show()

In [6]:
# tdir = "/home/hyoung/devel/vision_playground/test/iai/obstacles"
# tdir = "/home/hunter/data/images/test_cases"
tdir = "/home/hyoung/Documents/rosbags/extracted/depth"

In [7]:
found_imgs, _ = grab_dir_images(tdir)
_imgs = widgets.Select(
                        options=sorted(found_imgs),
                        value=str(sorted(found_imgs)[0]),
                        description='File:',
                        layout=Layout(width='100%')
                    )

display(_imgs)

U2VsZWN0KGRlc2NyaXB0aW9uPXUnRmlsZTonLCBsYXlvdXQ9TGF5b3V0KHdpZHRoPXUnMTAwJScpLCBvcHRpb25zPSgnL2hvbWUvaHlvdW5nL0RvY3VtZW50cy9yb3NiYWdzL2V4dHJhY3RlZC/igKY=


In [9]:
testimg = cv2.imread(_imgs.value, cv2.IMREAD_GRAYSCALE)
plt.figure(1)
plt.imshow(testimg)
plt.show()

In [10]:
from utils.uv_mapping_utils import *
from utils.seg_utils import *

_img = _imgs.value
img = cv2.imread(_img, cv2.IMREAD_GRAYSCALE)

kernelI = cv2.getStructuringElement(cv2.MORPH_RECT,(10,10))
img = cv2.morphologyEx(img, cv2.MORPH_CLOSE, kernelI)

# umap, vmapMasked, img, obs, nObs, pxls,vmap,mask, maskInv, dt = uv_pipeline(_img, flag_displays=True)

umap1, vmap1,_ = get_uv_map(img)
umap2 = umapping(img);              vmap2 = vmapping(img);               overlay = make_uv_overlay(img,umap2,vmap2)
h,w = umap1.shape[:2]

raw_umap = np.copy(umap1)

# =================================================================================
# =================================================================================
# =================================================================================

dead_y = 10
deadzoneU = raw_umap[1:dead_y+1, :]
blankStrip = np.zeros((h-dead_y,w),dtype=np.uint8)

print(np.max(deadzoneU))
_, deadzoneU = cv2.threshold(deadzoneU, 95, 255,cv2.THRESH_BINARY)
cv2.rectangle(raw_umap,(0,0),(raw_umap.shape[1],dead_y),(0,0,0), cv2.FILLED)
cv2.rectangle(raw_umap,(0,raw_umap.shape[0]-dead_y),(raw_umap.shape[1],raw_umap.shape[0]),(0,0,0), cv2.FILLED)


deadzone_mask = np.concatenate((deadzoneU,blankStrip), axis=0)
try: deadzone_mask = cv2.cvtColor(deadzone_mask, cv2.COLOR_GRAY2BGR)
except: print("[WARNING] ------------  Unnecessary Deadzone Image Color Converting")

kernelD = cv2.getStructuringElement(cv2.MORPH_RECT,(50,5))
deadzone_mask = cv2.morphologyEx(deadzone_mask, cv2.MORPH_CLOSE, kernelD)

try: raw_umap = cv2.cvtColor(raw_umap, cv2.COLOR_GRAY2BGR)
except: print("[WARNING] ------------  Unnecessary raw_umap Image Color Converting")
umap = cv2.addWeighted(raw_umap, 1.0, deadzone_mask, 1.0, 0)

try: umap = cv2.cvtColor(umap, cv2.COLOR_BGR2GRAY)
except: print("[WARNING] ------------  Unnecessary stripU1 Image Color Converting")

stripsU = strip_image(umap, nstrips=4, horizontal_strips=False)

strips = np.copy(stripsU)
print(stripsU.shape, strips.shape)

# _, stripU1 = cv2.threshold(stripsU[0], 7, 255,cv2.THRESH_BINARY)
# _, stripU2 = cv2.threshold(stripsU[1], 25, 255,cv2.THRESH_BINARY)
# _, stripU3 = cv2.threshold(stripsU[2], 30, 255,cv2.THRESH_BINARY)
# _, stripU4 = cv2.threshold(stripsU[3], 40, 255,cv2.THRESH_BINARY)


# fig, axes = plt.subplots(1,4)
# axes[0].imshow(stripsU[0])
# axes[1].imshow(stripsU[1])
# axes[2].imshow(stripsU[2])
# axes[3].imshow(stripsU[3])
# fig.show()

color = (255,0,255)
borderb = np.ones((1,w,3),dtype=np.uint8); borderb[:] = color
borders = np.ones((h,1,3),dtype=np.uint8); borders[:] = color
# newUdisp = np.concatenate((stripU1,borderb,stripU2,borderb,stripU3,borderb,stripU4), axis=0)
# newUdisp = np.concatenate((stripU1,borders,stripU2,borders,stripU3,borders,stripU4), axis=1)


# cv2.rectangle(raw_vmap,(0,0),(dead_x, raw_vmap.shape[0]),(0,0,0), cv2.FILLED)
# cv2.rectangle(raw_vmap,(raw_vmap.shape[1]-dead_x,0),(raw_vmap.shape[1],raw_vmap.shape[0]),(0,0,0), cv2.FILLED)
# newVdisp = np.concatenate((stripV1,borders,stripV2,borders,stripV3,borders,stripV4), axis=1)

_, tmp = cv2.threshold(umap, 10, 255,cv2.THRESH_TOZERO)

plt.figure(8)
plt.imshow(img)
plt.show()

plt.figure(10)
plt.imshow(umap)
plt.show()

plt.figure(11)
plt.imshow(vmap1)
plt.show()


plt.figure(12)
plt.imshow(umap2)
plt.show()

plt.figure(13)
plt.imshow(vmap2)
plt.show()

------------------------------------
[U-Mapping] Input Image (480 x 640):
------------------------------------
   * Un-Normalized Disparity Limits      = (0, 255)
   * Normalized Disparity Limits         = (-0.500, 4.415)
   * U-Map Disparity Limits              = (0.000, 254.000)
   * Un-Normalized Statistics   (μ, std) = (25.951, 51.875)
   * Normalized Statistics      (μ, std) = (-0.000, 1.000)
   * U-Map Statistics           (μ, std) = (1.882, 12.522)


ValueError: autodetected range of [nan, nan] is not finite

In [ ]:
norm = (vmap1 - np.mean(vmap1))/ np.std(vmap1)

plt.figure(14)
plt.imshow(norm)
plt.show()

In [ ]:
h,w = vmap.shape[:2]
# black = np.zeros((h,w,3),dtype=np.uint8)
black = np.copy(vmap)
black = cv2.cvtColor(black,cv2.COLOR_GRAY2BGR)

print(pxls[-1,0],pxls)

# ploty = np.linspace(pxls[0,1], pxls[-1,1])
plotx = np.linspace(pxls[0,0], pxls[-1,0])
try: # Try fitting polynomial nonzero data
    fit = np.poly1d(np.polyfit(pxls[:,0],pxls[:,1], 2))
    # Generate x and y values for plotting
#     plotx = fit(ploty)
    ploty = fit(plotx)
except:
    print("ERROR: Function 'polyfit' failed!")
    fit = [0, 0]; plotx = [0, 0]
# xs = np.asarray(ploty,dtype=np.int32)
# ys = np.asarray(plotx,dtype=np.int32)

xs = np.asarray(plotx,dtype=np.int32)
ys = np.asarray(ploty,dtype=np.int32)

pts = np.vstack(([xs],[ys])).transpose()
cv2.polylines(black, [pts], 0, (255,0,255),5)
plt.figure(9)
plt.imshow(black)
# plt.plot(pxls[:,0],pxls[:,1])
# plt.plot(ploty,-plotx)

plt.show()

In [3]:
def tester2(_img, e1=1, e2=5,tmpThresh=3,greyThreshU=7,greyThreshV=5,cnt_thresh=100.0,
                 show_helpers=True,use_umap=True, flip_thresh_bin=True, norm_grey_thresh=0.5):
    img = cv2.imread(_img, cv2.IMREAD_GRAYSCALE)
    pipe.img = img
    plt.figure(0)
    plt.imshow(img)
    plt.show()

    if(flip_thresh_bin):
        umap, vmap, overlay = pipe.get_uv_map(img, get_overlay=True)
        dead_x = 2; dead_y = 10

        cv2.rectangle(umap,(0,0),(umap.shape[1],dead_y),(0,0,0), cv2.FILLED)
        cv2.rectangle(umap,(0,umap.shape[0]-dead_y),(umap.shape[1],umap.shape[0]),(0,0,0), cv2.FILLED)

        cv2.rectangle(vmap,(0,0),(dead_x, vmap.shape[0]),(0,0,0), cv2.FILLED)
        cv2.rectangle(vmap,(vmap.shape[1]-dead_x,0),(vmap.shape[1],vmap.shape[0]),(0,0,0), cv2.FILLED)
    else:
        umap = umapping(img)
        vmap = vmapping(img)
        overlay = make_uv_overlay(img,umap,vmap)


    h,w = umap.shape[:2]
    clone = np.copy(umap)
#     cv2.rectangle(clone,(0,0),(w,10),(0,0,0), cv2.FILLED)
#     cv2.rectangle(clone,(0,h-10),(w,h),(0,0,0), cv2.FILLED)
    print(np.max(clone), np.min(clone))



    _,tmp = cv2.threshold(clone,greyThreshU,np.max(clone),cv2.THRESH_BINARY)
    opening = tmp
#     _,tmp = cv2.threshold(clone,greyThreshU,np.max(clone),cv2.THRESH_TOZERO)

#     cv2.rectangle(tmp,(0,0),(w,10),(0,0,0), cv2.FILLED)
#     cv2.rectangle(tmp,(0,h-10),(w,h),(0,0,0), cv2.FILLED)

    kernel = cv2.getStructuringElement(cv2.MORPH_RECT,(e1,e2))
#     opening = cv2.morphologyEx(tmp, cv2.MORPH_CLOSE, kernel)
#     _,opening = cv2.threshold(opening,tmpThresh,np.max(clone),cv2.THRESH_BINARY)

    _,greyV = cv2.threshold(vmap,greyThreshV,np.max(vmap),cv2.THRESH_BINARY)

    _, contours, hierarchy = cv2.findContours(opening,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
    areas = [cv2.contourArea(cnt) for cnt in contours]
    perimeters = [cv2.arcLength(cnt,True) for cnt in contours]

    filtered_cnts = [cnt for cnt in contours if cv2.arcLength(cnt,True) >= cnt_thresh]
    filtered_perimeters = [cv2.arcLength(cnt,True) for cnt in filtered_cnts]
    print("Filtered Contour Perimeters:",filtered_perimeters)

    display = cv2.cvtColor(opening, cv2.COLOR_GRAY2BGR)
    for cnt in filtered_cnts:
        cv2.drawContours(display, [cnt], 0, (255,0,0), 2)

    cnts, limits, ellipses, pts, centers = pipe.extract_contour_x_bounds(filtered_cnts)

    nObs = len(pipe.disparityBounds)
    obsImgs = []
    disp = np.copy(img)
    display_obstacles = cv2.cvtColor(disp,cv2.COLOR_GRAY2BGR)

    for i in range(nObs):
        tmp = np.copy(img)
        tmp = cv2.cvtColor(tmp,cv2.COLOR_GRAY2BGR)
        try:
            xs = pipe.xBounds[i]
            ys,disp = pipe.find_obstacles(greyV,pipe.disparityBounds[i], _threshold=30, lower_threshold=1, verbose = True, display=greyV)
            cv2.rectangle(tmp,(xs[0],ys[0]),(xs[1],ys[-1]),(150,0,0),1)
            cv2.rectangle(display_obstacles,(xs[0],ys[0]),(xs[1],ys[-1]),(150,0,0),1)
        except: pass
        obsImgs.append(tmp)


    plt.figure(1)
    plt.imshow(opening)
    plt.show()

    plt.figure(2)
    plt.imshow(overlay)
    plt.show()

    plt.figure(3)
    plt.imshow(display)
    plt.show()

    plt.figure(4)
    plt.imshow(display_obstacles)
    plt.show()

    plt.figure(6)
    plt.imshow(greyV)
    plt.show()

In [4]:
vboat.dead_x = 0
vboat.dead_y = 5

In [5]:
def thresholding(_img, e1=1, e2=5,tmpThresh=70
                 
                 ,greyThreshU=7,greyThreshV=20,cnt_thresh=100.0,
                 show_helpers=True,use_umap=True, flip_thresh_bin=True, norm_grey_thresh=0.5
):
    if(show_helpers):
        vboat.pipeline(_img, threshU1=greyThreshU,threshU2=greyThreshV, threshV2=tmpThresh)
        display_obstacles = cv2.cvtColor(vboat.img, cv2.COLOR_GRAY2BGR)

        for ob in vboat.obstacles:
            cv2.rectangle(display_obstacles,ob[0],ob[1],(150,0,0),1)
        plt.figure(9)
        plt.imshow(display_obstacles)
        plt.show()
    else:
        tester2(_img,e1,e2,tmpThresh,greyThreshU,greyThreshV,cnt_thresh,show_helpers,use_umap,flip_thresh_bin, norm_grey_thresh)
    
        

In [6]:
# found_imgs, _ = grab_dir_images("/home/hyoung/devel/vision_playground/test/iai/obstacles")
# found_imgs, _ = grab_dir_images("/home/hunter/data/images/test_cases")
# found_imgs, _ = grab_dir_images("/home/hunter/devel/vision_playground/test/frames")
# found_imgs, _ = grab_dir_images("/home/hyoung/Documents/rosbags/extracted/depth")
found_imgs, _ = grab_dir_images("/home/hunter/data/rosbags/extracted/depth")
ttt = interactive(thresholding,_img = widgets.Select(
                        options=sorted(found_imgs),
                        value=str(sorted(found_imgs)[0]),
                        description='File:',
                        layout=Layout(width='100%')
                    ), e1 = (1,255,1), e2 = (1,255,1),
    tmpThresh = (0,255,1), greyThreshU = (0.0,255.0,0.1), greyThreshV = (0,255,1), cnt_thresh = (0,5000,100),
    show_helpers = True, use_umap = True, flip_thresh_bin = False, norm_grey_thresh = (0.0,1.0,0.01)
)
display(ttt)

aW50ZXJhY3RpdmUoY2hpbGRyZW49KFNlbGVjdChkZXNjcmlwdGlvbj11J0ZpbGU6JywgbGF5b3V0PUxheW91dCh3aWR0aD11JzEwMCUnKSwgb3B0aW9ucz0oJy9ob21lL2h1bnRlci9kYXRhL3LigKY=


In [14]:
tImg = np.copy(vboat.img)

plt.figure(0)
plt.imshow(tImg)
plt.show()

plt.figure(1)
plt.imshow(vboat.umap_raw)
plt.show()

# plt.figure(2)
# plt.imshow(vboat.vmap_raw)
# plt.show()



raw_umap, raw_vmap, dt = vboat.get_uv_map(tImg)

deadzoneU = raw_umap[1:5+1, :]
_, deadzoneU = cv2.threshold(deadzoneU, 95, 255,cv2.THRESH_BINARY)



cv2.rectangle(raw_umap,(0,0),(raw_umap.shape[1],vboat.dead_y),(0,0,0), cv2.FILLED)
cv2.rectangle(raw_umap,(0,raw_umap.shape[0]-vboat.dead_y),(raw_umap.shape[1],raw_umap.shape[0]),(0,0,0), cv2.FILLED)
cv2.rectangle(raw_vmap,(0,0),(vboat.dead_x, raw_vmap.shape[0]),(0,0,0), cv2.FILLED)
cv2.rectangle(raw_vmap,(raw_vmap.shape[1]-vboat.dead_x,0),(raw_vmap.shape[1],raw_vmap.shape[0]),(0,0,0), cv2.FILLED)


_, stripU1 = cv2.threshold(vboat.stripsU_raw[0], 7, 255,cv2.THRESH_BINARY)
_, stripU2 = cv2.threshold(vboat.stripsU_raw[1], 20, 255,cv2.THRESH_BINARY)
_, stripU3 = cv2.threshold(vboat.stripsU_raw[2], 30, 255,cv2.THRESH_BINARY)
_, stripU4 = cv2.threshold(vboat.stripsU_raw[3], 40, 255,cv2.THRESH_BINARY)
_, stripU5 = cv2.threshold(vboat.stripsU_raw[4], 40, 255,cv2.THRESH_BINARY)
_, stripU6 = cv2.threshold(vboat.stripsU_raw[5], 40, 255,cv2.THRESH_BINARY)

plt.figure(4)
plt.imshow(stripU1)
plt.show()

kernel = cv2.getStructuringElement(cv2.MORPH_RECT,(10,2))
stripU1 = cv2.morphologyEx(stripU1, cv2.MORPH_CLOSE, kernel)
stripU2 = cv2.morphologyEx(stripU2, cv2.MORPH_CLOSE, kernel)
stripU3 = cv2.morphologyEx(stripU3, cv2.MORPH_OPEN, kernel)

hUs = vboat.stripsU_raw[0].shape[0]
blankStrip = np.zeros((hUs-vboat.dead_y,vboat.w),dtype=np.uint8)
deadzone_mask = np.concatenate((deadzoneU,blankStrip), axis=0)
try: deadzone_mask = cv2.cvtColor(deadzone_mask, cv2.COLOR_GRAY2BGR)
except: print("[WARNING] ------------  Unnecessary Deadzone Image Color Converting")

kernelD = cv2.getStructuringElement(cv2.MORPH_RECT,(50,5))
deadzone_mask = cv2.morphologyEx(deadzone_mask, cv2.MORPH_CLOSE, kernelD)


    
stripU1 = cv2.addWeighted(stripU1, 1.0, deadzone_mask, 1.0, 0)
   
umap = np.concatenate((stripU1,stripU2,stripU3,stripU4,stripU5,stripU6), axis=0)




# plt.figure(3)
# plt.imshow(raw_vmap)
# plt.show()










In [11]:
dispU1,dispU2,compU = vboat.umap_displays()
dispV1,dispV2,compV = vboat.vmap_displays()

plt.figure(6)
plt.imshow(vboat.img)
plt.show()

plt.figure(0)
plt.imshow(dispU1)
plt.show()

plt.figure(1)
plt.imshow(dispU2)
plt.show()

# plt.figure(2)
# plt.imshow(compU)
# plt.show()

plt.figure(3)
plt.imshow(dispV1)
plt.show()

plt.figure(4)
plt.imshow(dispV2)
plt.show()

# plt.figure(5)
# plt.imshow(compV)
# plt.show()

In [ ]:
disp = np.copy(vboat.umap_raw)
col = cv2.cvtColor(disp,cv2.COLOR_GRAY2BGR)
col = cv2.cvtColor(col,cv2.COLOR_BGR2Luv)

plt.figure(1)
plt.imshow(col)
plt.show()

In [68]:
ellipses = []
disp = np.copy(vboat.umap_processed)
disp = cv2.cvtColor(disp,cv2.COLOR_GRAY2BGR)
for cnt in vboat.filtered_contours:
    ellipse = cv2.fitEllipse(cnt)
    ellipses.append(ellipse)
    cv2.ellipse(disp,ellipse,(0,255,255),1)
    
plt.figure(1)
plt.imshow(disp)
plt.show()

In [69]:
img = np.copy(vboat.img)
print(vboat.obstacles[0])
win = vboat.obstacles[0]
print(win[0][0], win[1][0])

rectsTypeA = [] # x0, y0, x1, y1
rectsTypeB = [] # x, y, w, h
for win in vboat.obstacles:
    
    wid = win[1][0] - win[0][0]
    h = win[1][1] - win[0][1]
    cx = win[0][0] + wid/2
    cy = win[0][1] + h/2
    
    tmpA = [win[0][0], win[0][1], win[1][0], win[1][1]]
    tmpB = [cx, cy, wid, h]
    rectsTypeA.append(tmpA)
    rectsTypeB.append(tmpB)


# cv2.circle(img,(pxl[0],pxl[1]),2,(255,0,255),-1)

print(wid, h)
print(cx, cy)
print(rectsTypeB)
print(vboat.nObs)

[(443, 240), (559, 280)]
(443, 559)
(562, 240)
(281, 160)
[[501, 260, 116, 40], [281, 160, 562, 240]]
2


In [70]:
def test_intersection(boxA, boxB):
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])
    
    
    interArea = max(0, xB - xA + 1) * max(0, yB - yA + 1)
    boxAArea = (boxA[2] - boxA[0] + 1) * (boxA[3] - boxA[1] + 1)
    boxBArea = (boxB[2] - boxB[0] + 1) * (boxB[3] - boxB[1] + 1)
    iou = interArea / float(boxAArea + boxBArea - interArea)
    return iou

def union(a,b):
    x = min(a[0], b[0])
    y = min(a[1], b[1])
    w = max(a[0]+a[2], b[0]+b[2]) - x
    h = max(a[1]+a[3], b[1]+b[3]) - y
    return (x, y, w, h)

In [71]:
n = vboat.nObs
idcs = []
pick = []
boxes = np.array(rectsTypeA)
x1 = boxes[:,0]
y1 = boxes[:,1]
x2 = boxes[:,2]
y2 = boxes[:,3]
area = (x2 - x1 + 1) * (y2 - y1 + 1)
idxs = np.argsort(y2)
overlapThresh = 0.05
supressed = []
top = []
non = []
print(idxs)
while len(idxs) > 0:
    # grab the last index in the indexes list, add the index
    # value to the list of picked indexes, then initialize
    # the suppression list (i.e. indexes that will be deleted)
    # using the last index
    last = len(idxs) - 1
    i = idxs[last]
    pick.append(i)
    suppress = [last]
    # loop over all indexes in the indexes list
    print(i)
    overlaps = False
    for pos in xrange(0, last):
        # grab the current index
        j = idxs[pos]

        # find the largest (x, y) coordinates for the start of
        # the bounding box and the smallest (x, y) coordinates
        # for the end of the bounding box
        xx1 = max(x1[i], x1[j])
        yy1 = max(y1[i], y1[j])
        xx2 = min(x2[i], x2[j])
        yy2 = min(y2[i], y2[j])

        # compute the width and height of the bounding box
        w = max(0, xx2 - xx1 + 1)
        h = max(0, yy2 - yy1 + 1)

        # compute the ratio of overlap between the computed
        # bounding box and the bounding box in the area list
        
        overlap = float(w * h) / area[j]

        print("Window %d Overlaps %.4f with Rectangle %d" % (j,overlap, i) )
        # if there is sufficient overlap, suppress the
        # current bounding box
        if overlap > overlapThresh:
#             suppress.append(pos)
            supressed.append(pos)
            top.append([pos,i])
            overlaps = True

    # delete all indexes from the index list that are in the
    # suppression list
    print("suppressing idxs", suppress)
#     supressed.append(suppress)
    idxs = np.delete(idxs, suppress)
    if(not overlaps):
        non.append(last)

rectangles = [] 
for a,b in top:
    rect = union(rectsTypeA[a],rectsTypeA[b])
    rectangles.append(rect)
    

print(top)
print(non)
print(supressed)
mask = np.isin(non, supressed, invert=True)
remain = np.where(mask)
print(remain)
rectangles.append(rectsTypeA[remain[0][0]])

print(rectangles)
for i in range(n):
    has_overlap = False
    for j in range(n):
        if(i is not j):
            iou = test_intersection(rectsTypeA[i],rectsTypeA[j])
            if(iou >= 0.05):
                has_overlap = True
                idcs.append([i, j])
                print("IoU for Rect %d and Rect %d = %.3f" % (i, j, iou))
    if(not has_overlap):
        print("Rect %d has no overlapping segments" % i)

[0 1]
1
Window 0 Overlaps 1.0000 with Rectangle 1
('suppressing idxs', [1])
0
('suppressing idxs', [0])
[[0, 1]]
[0]
[0]
(array([], dtype=int64),)


IndexError: index 0 is out of bounds for axis 0 with size 0

In [72]:
disp = np.copy(vboat.img)
disp = cv2.cvtColor(disp,cv2.COLOR_GRAY2BGR)
for x1,y1,x2,y2 in boxes[pick]:
    cv2.rectangle(disp,(x1,y1),(x2,y2),(0,255,255), 1)
    
plt.figure(10)
plt.imshow(disp)
plt.show()

In [77]:
disp = np.zeros_like(vboat.img)
disp = cv2.cvtColor(disp,cv2.COLOR_GRAY2BGR)
masks = []
for win in vboat.obstacles:
    mask = np.copy(disp)
    cv2.rectangle(mask,win[0],win[1],(255,255,255), cv2.FILLED)
    mask = cv2.cvtColor(mask,cv2.COLOR_BGR2GRAY)
    masks.append(mask)
    

tmp = cv2.bitwise_and(vboat.img,vboat.img, mask=masks[0])

tmpMean = np.mean(tmp)
print(tmpMean/0.001)
plt.figure(2)
plt.imshow(tmp)
plt.show()

181.435546875


In [95]:
%load_ext autoreload
%autoreload 2
import time
import numpy as np
import cv2, Image
import pyrealsense as pyrs
from pyrealsense.constants import rs_option

from tools.r200_camera import CameraR200

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [16]:
serv = pyrs.Service()
dev = serv.Device(depth_control_preset=1)

In [42]:
try:  # set custom gain/exposure values to obtain good depth image
    custom_options = [(rs_option.RS_OPTION_R200_LR_EXPOSURE, 100.0),
                      (rs_option.RS_OPTION_R200_LR_GAIN, 100.0)]
    dev.set_device_options(*zip(*custom_options))
except pyrs.RealsenseError:
    pass  # options are not available on all devices

In [56]:
dev.apply_ivcam_preset(0)
intr = dev.depth_intrinsics
extr = dev.get_device_extrinsics(2,3)
print(extr.rotation)
Fx = intr.fx
Fy = intr.fy
baseline = extr.translation[0]
dscale = dev.depth_scale
print(Fx,baseline,dscale)

cnt = 0
last = time.time()
smoothing = 0.9
fps_smooth = 30


(580.9887084960938, -0.07007873058319092, 0.0010000000474974513)


In [53]:
dev.wait_for_frames()
c = dev.color
cc = cv2.cvtColor(c, cv2.COLOR_RGB2BGR)

plt.figure(0)
plt.imshow(c)
plt.show()

d_raw = dev.depth * dev.depth_scale
print(np.max(dev.depth))
print(np.max(d_raw))

plt.figure(1)
plt.imshow(dev.depth * dev.depth_scale)
plt.show()

d_raw = d_raw.astype(np.uint8)
norm_image = cv2.normalize(d_raw, None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)
norm_image = cv2.cvtColor(norm_image, cv2.COLOR_GRAY2RGB)
depth_raw = cv2.applyColorMap(d_raw, cv2.COLORMAP_RAINBOW)

# plt.figure(0)
# plt.imshow(d_raw)
# plt.show()

65535
65.53500311274547


In [94]:
dev.stop()
serv.stop()
r200.__del__()

[INFO] Closing CameraR200 object


In [96]:
r200 = CameraR200()

[INFO] Closing CameraR200 object


In [81]:
intr = r200.get_intrinsics()

In [82]:
pp.pprint(intr.ppy)

238.56268310546875


In [ ]:
r200.loop()

[1551581645.175] R200 FPS = 54.26
[1551581645.507] R200 FPS = 51.85
[1551581645.843] R200 FPS = 49.63
[1551581646.172] R200 FPS = 47.71
[1551581646.503] R200 FPS = 45.96
[1551581646.877] R200 FPS = 44.04
[1551581647.201] R200 FPS = 42.72
[1551581647.535] R200 FPS = 41.44
[1551581647.868] R200 FPS = 40.30
[1551581648.203] R200 FPS = 39.25
[1551581648.543] R200 FPS = 38.27
[1551581648.872] R200 FPS = 37.48
[1551581649.207] R200 FPS = 36.72
[1551581649.540] R200 FPS = 36.05
[1551581649.876] R200 FPS = 35.42
[1551581650.207] R200 FPS = 34.90
[1551581650.534] R200 FPS = 34.47
[1551581650.870] R200 FPS = 34.00
[1551581651.201] R200 FPS = 33.62
[1551581651.538] R200 FPS = 33.22
[1551581651.876] R200 FPS = 32.86
[1551581652.200] R200 FPS = 32.66
[1551581652.539] R200 FPS = 32.35
[1551581652.876] R200 FPS = 32.08
[1551581653.208] R200 FPS = 31.88
[1551581653.534] R200 FPS = 31.76
[1551581653.876] R200 FPS = 31.51
[1551581654.203] R200 FPS = 31.42
[1551581654.535] R200 FPS = 31.28
[1551581654.86